In [1]:
# Homework 3
#
# This homework assignment is for the practice section of homework 3 at the University of Utah in CS6350 Machine Learning.
# This code will covering:
#    1. Standard Perceptron
#    2. Voted Perceptron
#    3. Average Perceptron
#    4. Comparing the average prediction errors for the three methods
#
# @author  Scott Merkley
# @version November 01, 2024

# **Import Libraries**

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

# **Perceptron Class**

In [23]:
class Perceptron:
  def __init__(self, learning_rate = 0.1, n_epochs = 10):
    self.lr       = learning_rate
    self.n_epochs = n_epochs
    self.weights  = None
    self.bias     = None


  # This helper method initializes the perceptron by finding the number of rows, number of columns, and returning y as an
  # array that contains {-1, 1} instead of {0, 1}. By changing y, you can easily calculate the new weights by just multiplying
  # your learning rate by y[i] instead of having to do math behind the scenes.
  #
  def initialize_perceptron(self, X, y):
    n_rows, n_columns = X.shape
    y = np.where(y <= 0, -1, 1) # Converting labels to -1 or 1 so it is easier to compute weights and bias
    return n_rows, n_columns, y


  # Standard Train trains the model using the standard perceptron model. This first initializes the weights and bias, then
  # runs the dataset through a number of epochs, calculating predictions and then updating the weights to get a better
  # calculation and predictions when predicting values.
  #
  def standard_train(self, X, y):
    n_rows, n_columns, y = self.initialize_perceptron(X, y)
    self.weights = np.zeros(n_columns) # Initializing weights to zero
    self.bias = 0                      # This is just a term that tags along on the end

    for epoch in range(self.n_epochs + 1):
      for i, x_i in enumerate(X):
        prediction = x_i @ self.weights + self.bias
        y_pred     = np.where(prediction >= 0, 1, 0) # Change the prediction to 1 if there is anything

        # Update Weights
        if y[i] * y_pred <= 0: # The Point is Misclassified
          self.weights += self.lr * y[i] * x_i
          self.bias    += self.lr * y[i]

      self.update_user(epoch)


  # Voted Train trains the model using the voted perceptron model. This first initializes the weights, bias, margins, and voted
  # weights and bias then runs the dataset through a number of epochs, calculating predictions and then updating the weights while
  # saving the margins to get a better calculation and predictions when predicting values.
  #
  def voted_train(self, X, y):
    n_rows, n_columns, y = self.initialize_perceptron(X, y)
    self.weights  = []
    self.bias     = []
    margins       = [] # The number of predictions made by weights_m
    margin        = 0  # Initialize margin to zero
    voted_weights = np.zeros(n_columns)
    voted_bias    = 0

    for epoch in range(self.n_epochs + 1):
      for i, x_i in enumerate(X):
        prediction = x_i @ voted_weights + voted_bias
        y_pred = np.where(prediction >= 0, 1, 0)

        # Update Weights:
        # The Point is Misclassified
        if prediction <= 0:
          self.weights.append(voted_weights.copy()) # Store the current weights
          self.bias.append(voted_bias)
          margins.append(margin)

          # Update the weights
          voted_weights += self.lr * y[i] * x_i
          voted_bias    += self.lr * y[i]
          margin         = 1
        else:
          margin += 1

      self.update_user(epoch)

    # Store the last weights
    self.weights.append(voted_weights.copy())
    self.bias.append(voted_bias)
    margins.append(margin)

    # Set the final weights
    total_counts = sum(margins)
    self.weights = sum(c * w for c, w in zip(margins, self.weights)) / total_counts
    self.bias    = sum(c * b for c, b in zip(margins, self.bias)) / total_counts


  # Averaged Train trains the model using the averaged perceptron model. This first initializes the weights, bias, and a.
  # 'a' is an average term that adds the newly calculated weight to itself everytime the weights are recalculated. Then
  # runs the dataset through a number of epochs, calculating predictions and then updating the weights while saving the
  # margins to get a better calculation and predictions when predicting values.
  #
  def averaged_train(self, X, y):
    n_rows, n_columns, y = self.initialize_perceptron(X, y)
    self.weights = np.zeros(n_columns) # Initializing weights to zero
    self.bias = 0                      # This is just a term that tags along on the end
    a = 0

    for epoch in range(self.n_epochs + 1):
      for i, x_i in enumerate(X):
        prediction = x_i @ self.weights + self.bias
        y_pred     = np.where(prediction >= 0, 1, 0) # Change the prediction to 1 if there is anything

        # Update Weights
        if y[i] * y_pred <= 0: # The Point is Misclassified
          self.weights += self.lr * y[i] * x_i
          self.bias    += self.lr * y[i]

        a += self.weights
      self.update_user(epoch)

    # Set the final weights
    self.weights = a


  # Prints out 'Current epoch: #' to the user in steps but will only print out 10 total steps depending
  # on the number of epochs initialized in the perceptron.
  #
  def update_user(self, epoch):
    # Visually Update
    if epoch % (self.n_epochs // 10) == 0:
      print(f'Current epoch: {epoch}')


  # Returns the weights and bias to the user.
  #
  def get_weights(self):
      return self.weights, self.bias


  # Predicts the next value given a test example X.
  #
  def predict(self, X):
    prediction = X @ self.weights + self.bias
    return np.where(prediction >= 0, 1, 0)

# **Import Bank Note Dataset**

In [19]:
columns = ["Variance", "Skewness", "Curtosis", "Entropy", "Label"]
bank_note_train = pd.read_csv('https://drive.google.com/uc?export=download&id=1Ua6KGAIzqVb6v6iYtmyhbYWhfrkGTjl8', names = columns)
bank_note_test = pd.read_csv('https://drive.google.com/uc?export=download&id=1jSPTIRkWYR4nRCWTN8Mj-NBXAgWBBxmU', names = columns)
bank_note_train

,Variance,Skewness,Curtosis,Entropy,Label
0,3.848100,10.15390,-3.85610,-4.22280,0
1,4.004700,0.45937,1.36210,1.61810,0
2,-0.048008,-1.60370,8.47560,0.75558,0
3,-1.266700,2.81830,-2.42600,-1.88620,1
4,2.203400,5.99470,0.53009,0.84998,0
...,...,...,...,...,...
867,0.273310,4.87730,-4.91940,-5.81980,1
868,1.063700,3.69570,-4.15940,-1.93790,1
869,-1.242400,-1.71750,-0.52553,-0.21036,1
870,1.837300,6.12920,0.84027,0.55257,0


In [31]:
# Standard Perceptron
p = Perceptron()
X_train = bank_note_train.drop('Label', axis = 1).values
y_train = bank_note_train['Label'].values
p.standard_train(X_train, y_train)

X_test = bank_note_test.drop('Label', axis = 1).values
y_test = bank_note_test['Label'].values

print('\nWeights:', p.weights)
print('Bias:', p.bias)
correct = 0
for i in range(len(X_test)):
  if y_test[i] == p.predict(X_test[i]):
    correct += 1
print(f"Percent Correct: {correct / len(X_test) * 100:0.3}%")
standard_prediction_error = correct / len(X_test)

Current epoch: 0
Current epoch: 1
Current epoch: 2
Current epoch: 3
Current epoch: 4
Current epoch: 5
Current epoch: 6
Current epoch: 7
Current epoch: 8
Current epoch: 9
Current epoch: 10

Weights [-1446.03416094 -1656.34032247  -774.22500229   144.00164218]
Bias -347.8000000000106
Percent Correct: 77.6%


In [32]:
# Voted Perceptron
p = Perceptron()
X_train = bank_note_train.drop('Label', axis = 1).values
y_train = bank_note_train['Label'].values
p.voted_train(X_train, y_train)

X_test = bank_note_test.drop('Label', axis = 1).values
y_test = bank_note_test['Label'].values

print('\nWeights:', p.weights)
print('Bias:', p.bias)
correct = 0
for i in range(len(X_test)):
  if y_test[i] == p.predict(X_test[i]):
    correct += 1
print(f"Percent Correct: {correct / len(X_test) * 100:0.3}%")
voted_prediction_error = correct / len(X_test)

Current epoch: 0
Current epoch: 1
Current epoch: 2
Current epoch: 3
Current epoch: 4
Current epoch: 5
Current epoch: 6
Current epoch: 7
Current epoch: 8
Current epoch: 9
Current epoch: 10

Weights [-677.48667147 -933.28828289 -238.56477277   48.81357414]
Bias -143.50556713928037
Percent Correct: 73.6%


In [33]:
# Averaged Perceptron
p = Perceptron()
X_train = bank_note_train.drop('Label', axis = 1).values
y_train = bank_note_train['Label'].values
p.averaged_train(X_train, y_train)

X_test = bank_note_test.drop('Label', axis = 1).values
y_test = bank_note_test['Label'].values

print('\nWeights:', p.weights)
print('Bias:', p.bias)
correct = 0
for i in range(len(X_test)):
  if y_test[i] == p.predict(X_test[i]):
    correct += 1
print(f"Percent Correct: {correct / len(X_test) * 100:0.3}%")
averaged_prediction_error = correct / len(X_test)

Current epoch: 0
Current epoch: 1
Current epoch: 2
Current epoch: 3
Current epoch: 4
Current epoch: 5
Current epoch: 6
Current epoch: 7
Current epoch: 8
Current epoch: 9
Current epoch: 10

Weights [-6928360.11121192 -7883151.11055428 -3706297.63544075   686955.91233012]
Bias -347.8000000000106
Percent Correct: 78.6%


In [53]:
# Comparing the Prediction Errors
prediction_errors = pd.DataFrame([['Standard', standard_prediction_error], ['Voted', voted_prediction_error], ['Averaged', averaged_prediction_error]], columns = ['Algorithm', 'Prediction Error'])
prediction_errors.sort_values(by = 'Prediction Error', ascending = False)

,Algorithm,Prediction Error
2,Averaged,0.786
0,Standard,0.776
1,Voted,0.736
